# Data Preparation

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 15,8
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
# Creating imaginary customer data for our recommendation of the area

In [3]:
model_data = pd.read_csv('/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/Austin_House_data/train_nlp_dataset.csv', index_col=0)

In [4]:
pd.set_option('display.max_columns', None)
model_data

,new_price,garageSpaces,hasSpa,hasView,homeType,numPriceChanges,lotSizeSqFt,livingAreaSqFt,avgSchoolRating,numOfBathrooms,numOfBedrooms,numOfStories,numberOfSchools,rank,lat_long,sy yes,quiet cul,quiet cul de,en suite,en,zilker,win,steel,tankless water,washer dryer,floor,level,cul de lot,vault,cul,subway,barton,main level,default,plantation,walk shower,ton natural light,southpark,hike,construction,heart,de lot,sprinkler sy,gourmet kitchen,outdoor living,pit,laminate,concept,custom build,lamar,private,hike bike,fiber,curb,ut,country,south,natural,course,balcony,bright,shower,kitchen,formal,tile,outdoor,closet,vanity,mother law,brand,height,quartz,vinyl,living space,large corner,steiner,separate shower,shed,easy access,paint,breakfast,cul de,water heater,travis,bay,bamboo,heater,downtown,grill,crown molding,classic,restaurant,luxury,mid century,energy,lush,guest,wine,quartz counter,vaulted,shopping dining,stainless,community,view,bungalow,storage,marble,tech,oak,charm,detail,locate near,dining area,landscape,pool,walking,breakfast area,remodel,new hvac,near,meticulously,shower walk,cute,central,roof,spa,airport,recess,kitchen granite counter,carport,backyard cover,design,dining kitchen,suite,chef,efficient,spacious suite,round,hvac,porch,northwest,open living,living,shopping,wall,boast,freshly
0,421143.0,0.0,0.0,0.0,Single Family,7.0,8494.0,1665.0,2.333333,2.0,3.0,1.0,3.0,19,1381.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.165122,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.136271,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.139789,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.138204,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.141931,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.159391,0.0,0.0,0.246251
1,384348.0,0.0,0.0,1.0,Single Family,2.0,7143.0,1600.0,3.333333,2.0,3.0,1.0,3.0,11,1359.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.099856,0.0,0.000000,0.000000,0.154085,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.222042,0.0,0.0,0.0,0.218603,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.239265,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,470048.0,0.0,0.0,0.0,Single Family,4.0,10454.0,3001.0,7.666667,3.0,4.0,2.0,3.0,18,1415.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.074857,0.0,0.000000,0.145703,0.115509,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.161950,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.245540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.123548,0.0,0.00000,0.0,0.0,0.0,0.154903,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,376128.0,1.0,0.0,0.0,Single Family,3.0,4356.0,1689.0,4.333333,3.0,2.0,3.0,3.0,12,1340.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.193886,0.166914,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0

In [5]:
model_data.shape

(10054, 147)

In [6]:
# Loading the customer data

customer_data = pd.read_csv('/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/Austin_House_data/BankChurners.csv', index_col=0)

In [7]:
pd.set_option('display.max_columns', None)

In [8]:
customer_data.head(1)

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
CLIENTNUM,,,,,,,,,,,,,,,,,,,,,,
768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991


In [9]:
customer_data.shape

(10127, 22)

In [10]:
# keeping the data we are gonna make recommendation
customer_data = customer_data[['Customer_Age','Gender','Dependent_count','Education_Level','Marital_Status','Income_Category']]
pd.DataFrame.reset_index(customer_data, drop=True, inplace=True)

In [11]:
customer_data = customer_data.rename(columns={'Customer_Age':'Age', 'Dependent_count': 'Dependent_Count'})

In [12]:
# Dropping extra rows to match the data
customer_data = customer_data[:-73]

In [13]:
customer_data.Education_Level.unique()

array(['High School', 'Graduate', 'Uneducated', 'Unknown', 'College',
       'Post-Graduate', 'Doctorate'], dtype=object)

In [14]:
customer_data.Income_Category.unique()

array(['$60K - $80K', 'Less than $40K', '$80K - $120K', '$40K - $60K',
       '$120K +', 'Unknown'], dtype=object)

In [15]:
# For loop to rank the income

for index in customer_data.index:
    if customer_data.loc[index,'Income_Category']=='Less than $40K':
        customer_data.loc[index,'Income_Rank'] = 1
    elif customer_data.loc[index,'Income_Category']=='$40K - $60K':
        customer_data.loc[index,'Income_Rank'] = 2
    elif customer_data.loc[index,'Income_Category']=='$60K - $80K':
        customer_data.loc[index,'Income_Rank'] = 3
    elif customer_data.loc[index,'Income_Category']=='$80K - $120K':
        customer_data.loc[index,'Income_Rank'] = 4
    elif customer_data.loc[index,'Income_Category']=='$120K +':
        customer_data.loc[index,'Income_Rank'] = 5
    else:
        customer_data.loc[index,'Income_Rank'] = 0

In [16]:
customer_data

,Age,Gender,Dependent_Count,Education_Level,Marital_Status,Income_Category,Income_Rank
0,45,M,3,High School,Married,$60K - $80K,3.0
1,49,F,5,Graduate,Single,Less than $40K,1.0
2,51,M,3,Graduate,Married,$80K - $120K,4.0
3,40,F,4,High School,Unknown,Less than $40K,1.0
4,40,M,3,Uneducated,Married,$60K - $80K,3.0
...,...,...,...,...,...,...,...
10049,55,M,2,Graduate,Married,Less than $40K,1.0
10050,49,M,1,Uneducated,Single,$120K +,5.0
10051,37,M,4,High School,Single,$60K - $80K,3.0
10052,31,M,1,Post-Graduate,Single,$40K - $60K,2.0


In [17]:
# sort owners by income
customer_data_sorted = customer_data.sort_values(['Income_Rank'], ascending=True)

In [18]:
df1 = customer_data_sorted[customer_data_sorted['Income_Rank'] == 0] # unknown income range
df2 = customer_data_sorted[customer_data_sorted['Income_Rank'] != 0] # known ordered income range

In [19]:
df1.reset_index(drop=False, inplace=True)
df1.rename(columns={'index':'Owner_ID'}, inplace=True)

/home/sergei/anaconda3/envs/final_project/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [20]:
df2.reset_index(drop=False, inplace=True)
df2.rename(columns={'index':'Owner_ID'}, inplace=True)

In [21]:
df1.shape

(1109, 8)

In [22]:
df2.shape

(8945, 8)

In [23]:
# let's spread evenly df1 across df2 maintaining the order of the df2:
for i in range(df1.shape[0]):
    df2.loc[(i*8+0.5)] = df1.loc[i]

/home/sergei/anaconda3/envs/final_project/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [24]:
df2.reset_index(drop=False, inplace=True)
df2.sort_values(['index'], ascending=True, inplace=True)

<ipython-input-24-0fca86352e7e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.sort_values(['index'], ascending=True, inplace=True)


In [25]:
# Check that insertion of Unknown income_category went as planned
df2.head(12)

,index,Owner_ID,Age,Gender,Dependent_Count,Education_Level,Marital_Status,Income_Category,Income_Rank
0,0.0,7515,31,F,0,High School,Single,Less than $40K,1.0
8945,0.5,7310,48,F,2,Unknown,Single,Unknown,0.0
1,1.0,4392,43,F,4,High School,Unknown,Less than $40K,1.0
2,2.0,4377,41,F,3,Unknown,Unknown,Less than $40K,1.0
3,3.0,3805,46,F,3,Graduate,Unknown,Less than $40K,1.0
4,4.0,3825,53,F,0,Graduate,Single,Less than $40K,1.0
5,5.0,3277,45,F,4,Unknown,Married,Less than $40K,1.0
6,6.0,7489,52,F,3,Graduate,Single,Less than $40K,1.0
7,7.0,7494,45,F,2,Post-Graduate,Married,Less than $40K,1.0
8,8.0,3823,44,F,4,Uneducated,Single,Less than $40K,1.0


In [26]:
# Finally ordered customer data with Unknown Income customers spreaded evenly across the dataframe
customer_data = df2.drop(['index','Income_Rank'], axis=1)
customer_data.reset_index(drop=True, inplace=True)

In [27]:
customer_data.head()

,Owner_ID,Age,Gender,Dependent_Count,Education_Level,Marital_Status,Income_Category
0,7515,31,F,0,High School,Single,Less than $40K
1,7310,48,F,2,Unknown,Single,Unknown
2,4392,43,F,4,High School,Unknown,Less than $40K
3,4377,41,F,3,Unknown,Unknown,Less than $40K
4,3805,46,F,3,Graduate,Unknown,Less than $40K


In [28]:
# Sorting housing data by rank and price
model_data_sorted = model_data.sort_values(['rank', 'new_price'], ascending=True)
model_data_sorted.reset_index(drop=True, inplace=True)

In [29]:
# adding sorted customers by income to sorted data
model_customer_data = pd.concat([customer_data, model_data_sorted], axis=1)

In [30]:
# uploading areas ranked by average price per zipcode/area
zip_rank = pd.read_csv('/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/Austin_House_data/zipcodes_traindata_sorted_by_average_price.csv', index_col=0)

In [31]:
# Merging customers and areas based on rank
rank_model_customer_data = pd.merge(zip_rank, model_customer_data, on=['rank'], how='left')

In [32]:
rank_model_customer_data

,zipcode,average_zip_price,rank,Owner_ID,Age,Gender,Dependent_Count,Education_Level,Marital_Status,Income_Category,new_price,garageSpaces,hasSpa,hasView,homeType,numPriceChanges,lotSizeSqFt,livingAreaSqFt,avgSchoolRating,numOfBathrooms,numOfBedrooms,numOfStories,numberOfSchools,lat_long,sy yes,quiet cul,quiet cul de,en suite,en,zilker,win,steel,tankless water,washer dryer,floor,level,cul de lot,vault,cul,subway,barton,main level,default,plantation,walk shower,ton natural light,southpark,hike,construction,heart,de lot,sprinkler sy,gourmet kitchen,outdoor living,pit,laminate,concept,custom build,lamar,private,hike bike,fiber,curb,ut,country,south,natural,course,balcony,bright,shower,kitchen,formal,tile,outdoor,closet,vanity,mother law,brand,height,quartz,vinyl,living space,large corner,steiner,separate shower,shed,easy access,paint,breakfast,cul de,water heater,travis,bay,bamboo,heater,downtown,grill,crown molding,classic,restaurant,luxury,mid century,energy,lush,guest,wine,quartz counter,vaulted,shopping dining,stainless,community,view,bungalow,storage,marble,tech,oak,charm,detail,locate near,dining area,landscape,pool,walking,breakfast area,remodel,new hvac,near,meticulously,shower walk,cute,central,roof,spa,airport,recess,kitchen granite counter,carport,backyard cover,design,dining kitchen,suite,chef,efficient,spacious suite,round,hvac,porch,northwest,open living,living,shopping,wall,boast,freshly
0,78725.0,2.526651e+05,1,7515,31,F,0,High School,Single,Less than $40K,114424.0,2.0,0.0,1.0,Single Family,2.0,8145.0,1287.0,3.000000,2.0,3.0,1.0,3.0,1411.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.129333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.123665,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.104154,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,78725.0,2.526651e+05,1,7310,48,F,2,Unknown,Single,Unknown,132039.0,0.0,0.0,1.0,Single Family,5.0,4660.0,2074.0,3.000000,3.0,5.0,2.0,3.0,1365.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.127492,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.197886,0.00000,0.0,0.0,0.068187,0.0,0.000000,0.000000,0.105216,0.130289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.109586,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.105678,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,78725.0,2.526651e+05,1,4392,43,F,4,High School,Unknown,Less than $40K,137189.0,0.0,0.0,0.0,Single Family,7.0,9452.0,1745.0,3.666667,2.0,4.0,1.0,3.0,1364.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0

In [33]:
# Counts of different areas to recommend
rank_model_customer_data.groupby(['zipcode']).size()

zipcode
78702.0    294
78703.0    188
78704.0    489
78705.0     39
78717.0    373
78721.0    170
78722.0     62
78723.0    402
78724.0    162
78725.0    121
78726.0    122
78727.0    294
78728.0    160
78729.0    221
78730.0     85
78731.0    314
78732.0    365
78733.0     86
78735.0    155
78736.0    103
78737.0    313
78739.0    409
78741.0    157
78744.0    325
78745.0    724
78746.0     79
78747.0    272
78748.0    803
78749.0    582
78750.0    197
78751.0    136
78752.0    108
78753.0    315
78754.0    285
78756.0    104
78757.0    371
78758.0    296
78759.0    373
dtype: int64

In [34]:
# we can observe that the dataset is imbalanced

In [35]:
rank_model_customer_data.Marital_Status.unique()

array(['Single', 'Unknown', 'Married', 'Divorced'], dtype=object)

In [36]:
rank_model_customer_data.Income_Category.unique()

array(['Less than $40K', 'Unknown', '$40K - $60K', '$60K - $80K',
       '$80K - $120K', '$120K +'], dtype=object)

In [37]:
rank_model_customer_data.Education_Level.unique()

array(['High School', 'Unknown', 'Graduate', 'Post-Graduate',
       'Uneducated', 'College', 'Doctorate'], dtype=object)

In [38]:
# getting rid of Unknown in each category. For simplicity using the value of the previous row.
# Categories with unknowns - (Marital_Status, Income_Category, Education_Level)

In [39]:
for i in range(rank_model_customer_data.shape[0]):
    if rank_model_customer_data.Marital_Status[i] == 'Unknown':
        rank_model_customer_data.Marital_Status[i] = rank_model_customer_data.Marital_Status[i-1]

<ipython-input-39-d1e2fec905d8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rank_model_customer_data.Marital_Status[i] = rank_model_customer_data.Marital_Status[i-1]


In [40]:
for i in range(rank_model_customer_data.shape[0]):
    if rank_model_customer_data.Income_Category[i] == 'Unknown':
        rank_model_customer_data.Income_Category[i] = rank_model_customer_data.Income_Category[i-1]

<ipython-input-40-cde075f81c34>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rank_model_customer_data.Income_Category[i] = rank_model_customer_data.Income_Category[i-1]


In [41]:
for i in range(rank_model_customer_data.shape[0]):
    if rank_model_customer_data.Education_Level[i] == 'Unknown':
        rank_model_customer_data.Education_Level[i] = rank_model_customer_data.Education_Level[i-1]

<ipython-input-41-e56ed66a6ad0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rank_model_customer_data.Education_Level[i] = rank_model_customer_data.Education_Level[i-1]


In [42]:
# Final dataset to work with
recommender_dataset = rank_model_customer_data[['zipcode','average_zip_price','Age','Gender','Dependent_Count',
                                               'Education_Level','Marital_Status','Income_Category']]

In [43]:
recommender_dataset.dtypes

zipcode              float64
average_zip_price    float64
Age                    int64
Gender                object
Dependent_Count        int64
Education_Level       object
Marital_Status        object
Income_Category       object
dtype: object

In [44]:
rank_model_customer_data

,zipcode,average_zip_price,rank,Owner_ID,Age,Gender,Dependent_Count,Education_Level,Marital_Status,Income_Category,new_price,garageSpaces,hasSpa,hasView,homeType,numPriceChanges,lotSizeSqFt,livingAreaSqFt,avgSchoolRating,numOfBathrooms,numOfBedrooms,numOfStories,numberOfSchools,lat_long,sy yes,quiet cul,quiet cul de,en suite,en,zilker,win,steel,tankless water,washer dryer,floor,level,cul de lot,vault,cul,subway,barton,main level,default,plantation,walk shower,ton natural light,southpark,hike,construction,heart,de lot,sprinkler sy,gourmet kitchen,outdoor living,pit,laminate,concept,custom build,lamar,private,hike bike,fiber,curb,ut,country,south,natural,course,balcony,bright,shower,kitchen,formal,tile,outdoor,closet,vanity,mother law,brand,height,quartz,vinyl,living space,large corner,steiner,separate shower,shed,easy access,paint,breakfast,cul de,water heater,travis,bay,bamboo,heater,downtown,grill,crown molding,classic,restaurant,luxury,mid century,energy,lush,guest,wine,quartz counter,vaulted,shopping dining,stainless,community,view,bungalow,storage,marble,tech,oak,charm,detail,locate near,dining area,landscape,pool,walking,breakfast area,remodel,new hvac,near,meticulously,shower walk,cute,central,roof,spa,airport,recess,kitchen granite counter,carport,backyard cover,design,dining kitchen,suite,chef,efficient,spacious suite,round,hvac,porch,northwest,open living,living,shopping,wall,boast,freshly
0,78725.0,2.526651e+05,1,7515,31,F,0,High School,Single,Less than $40K,114424.0,2.0,0.0,1.0,Single Family,2.0,8145.0,1287.0,3.000000,2.0,3.0,1.0,3.0,1411.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.129333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.123665,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.104154,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,78725.0,2.526651e+05,1,7310,48,F,2,High School,Single,Less than $40K,132039.0,0.0,0.0,1.0,Single Family,5.0,4660.0,2074.0,3.000000,3.0,5.0,2.0,3.0,1365.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.127492,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.197886,0.00000,0.0,0.0,0.068187,0.0,0.000000,0.000000,0.105216,0.130289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.109586,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.105678,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,78725.0,2.526651e+05,1,4392,43,F,4,High School,Single,Less than $40K,137189.0,0.0,0.0,0.0,Single Family,7.0,9452.0,1745.0,3.666667,2.0,4.0,1.0,3.0,1364.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0

In [45]:
# Let's put list of different areas and average price of house per area in another table
zipcode_average_price_list = recommender_dataset[['zipcode','average_zip_price']].drop_duplicates()

In [46]:
# sorted average price per area/zipcode
zipcode_average_price_list.sort_values(by='zipcode', inplace=True)

In [47]:
zipcode_average_price_list.to_csv('/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/final_project_houseprice_git/zipcode_average_house_price_2021.csv')

In [48]:
# now we can drop average_zip_price from our dataset
recommender_dataset.drop(columns=['average_zip_price'], inplace=True)

/home/sergei/anaconda3/envs/final_project/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [49]:
# Final dataset for areas recommendation
recommender_dataset

,zipcode,Age,Gender,Dependent_Count,Education_Level,Marital_Status,Income_Category
0,78725.0,31,F,0,High School,Single,Less than $40K
1,78725.0,48,F,2,High School,Single,Less than $40K
2,78725.0,43,F,4,High School,Single,Less than $40K
3,78725.0,41,F,3,High School,Single,Less than $40K
4,78725.0,46,F,3,Graduate,Single,Less than $40K
...,...,...,...,...,...,...,...
10049,78703.0,46,M,2,College,Married,$120K +
10050,78703.0,46,M,2,Graduate,Married,$120K +
10051,78703.0,39,M,1,Graduate,Single,$120K +
10052,78703.0,55,M,2,High School,Divorced,$120K +


In [50]:
recommender_dataset.zipcode.value_counts(normalize=True) # proportions of different areas in the dataset

78748.0    0.079869
78745.0    0.072011
78749.0    0.057887
78704.0    0.048637
78739.0    0.040680
78723.0    0.039984
78759.0    0.037100
78717.0    0.037100
78757.0    0.036901
78732.0    0.036304
78744.0    0.032325
78753.0    0.031331
78731.0    0.031231
78737.0    0.031132
78758.0    0.029441
78702.0    0.029242
78727.0    0.029242
78754.0    0.028347
78747.0    0.027054
78729.0    0.021981
78750.0    0.019594
78703.0    0.018699
78721.0    0.016909
78724.0    0.016113
78728.0    0.015914
78741.0    0.015616
78735.0    0.015417
78751.0    0.013527
78726.0    0.012134
78725.0    0.012035
78752.0    0.010742
78756.0    0.010344
78736.0    0.010245
78733.0    0.008554
78730.0    0.008454
78746.0    0.007858
78722.0    0.006167
78705.0    0.003879
Name: zipcode, dtype: float64

# Modelling

In [51]:
dataset_X = recommender_dataset.drop(columns=['zipcode'])
dataset_y = recommender_dataset.zipcode

In [52]:
X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_y, test_size=0.3, random_state=100, stratify=dataset_y) # we have stratify parameter as target data is imbalanced

In [53]:
# compute the class weights because of imbalanced dataset
class_weight_train = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)

/home/sergei/anaconda3/envs/final_project/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[78702. 78703. 78704. 78705. 78717. 78721. 78722. 78723. 78724. 78725.
 78726. 78727. 78728. 78729. 78730. 78731. 78732. 78733. 78735. 78736.
 78737. 78739. 78741. 78744. 78745. 78746. 78747. 78748. 78749. 78750.
 78751. 78752. 78753. 78754. 78756. 78757. 78758. 78759.], y=8855    78756.0
7537    78739.0
3995    78721.0
9030    78731.0
5833    78723.0
         ...   
427     78744.0
5917    78737.0
2983    78745.0
8726    78733.0
4228    78741.0
Name: zipcode, Length: 7037, dtype: float64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [54]:
# creating a dictionary of weights
class_list = list(np.unique(y_train))
zip_iterator = zip(class_list, class_weight_train)
weight_dictionary = dict(zip_iterator)

In [55]:
# determine categorical features
categorical_ix = X_train.select_dtypes(include=['object']).columns

In [56]:
# transformer of categorical features
t = [('cat', OneHotEncoder(), categorical_ix)]
col_transform = ColumnTransformer(transformers=t)

In [57]:
model = RandomForestClassifier(class_weight = weight_dictionary)

In [58]:
# define the data preparation and modeling pipeline
pipeline = Pipeline(steps=[('prep',col_transform), ('m',model)])

In [59]:
pipeline.fit(X_train, y_train)
y_predict = pipeline.predict(X_test)

In [60]:
# View accuracy score
accuracy_score(y_test, y_predict) # obviously the accuracy score is very low for 3 main reasons: 
# imaginary dataset with overlaping data between classes, low number of datapoints per class on average, imbalanced classes

0.114683460391117

In [61]:
# The point of this side task is not to achieve the high accuracy score which is impossible in this case but to create a functioning recommendation app. 
# Let's assume that we achieved high accuracy score here and move on

# Retraining the model on the whole dataset

In [62]:
# compute the class weights because of imbalanced dataset
class_weight_data = class_weight.compute_class_weight('balanced',np.unique(dataset_y),dataset_y)

/home/sergei/anaconda3/envs/final_project/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass classes=[78702. 78703. 78704. 78705. 78717. 78721. 78722. 78723. 78724. 78725.
 78726. 78727. 78728. 78729. 78730. 78731. 78732. 78733. 78735. 78736.
 78737. 78739. 78741. 78744. 78745. 78746. 78747. 78748. 78749. 78750.
 78751. 78752. 78753. 78754. 78756. 78757. 78758. 78759.], y=0        78725.0
1        78725.0
2        78725.0
3        78725.0
4        78725.0
          ...   
10049    78703.0
10050    78703.0
10051    78703.0
10052    78703.0
10053    78703.0
Name: zipcode, Length: 10054, dtype: float64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [63]:
# creating a dictionary of weights
class_list = list(np.unique(dataset_y))
zip_iterator = zip(class_list, class_weight_data)
weight_dictionary = dict(zip_iterator)

In [64]:
# determine categorical features
categorical_ix = dataset_X.select_dtypes(include=['object']).columns

In [65]:
# transformer of categorical features
t = [('cat', OneHotEncoder(), categorical_ix)]
col_transform = ColumnTransformer(transformers=t)

In [66]:
# define the model
model = RandomForestClassifier(class_weight = weight_dictionary)

In [67]:
# define the data preparation and modeling pipeline
pipeline = Pipeline(steps=[('prep',col_transform), ('m',model)])

In [68]:
# fit the pipeline
pipeline.fit(dataset_X, dataset_y)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  Index(['Gender', 'Education_Level', 'Marital_Status', 'Income_Category'], dtype='object'))])),
                ('m',
                 RandomForestClassifier(class_weight={78702.0: 0.8999283924095954,
                                                      78703.0: 1.4073348264277716,
                                                      78704.0: 0.5410612420622107,
                                                      78705.0: 6.7840755735492575,
                                                      78717.0: 0.7093269366445605,
                                                      7872...
                                                      78733.0: 3.076499388004896,
                                                      78735.0: 1.706960950764007,
                                                      78736.0: 2.5687276443536025,
          

In [69]:
# saving trained model

from joblib import dump, load

dump(pipeline, '/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/final_project_houseprice_git/model_recommendation.joblib') 

['/home/sergei/Desktop/LighthouseLabs/Re_enter_bootcamp/Final_Project/final_project_houseprice_git/model_recommendation.joblib']